**load imageless models**

In [ ]:
import joblib
import tensorflow as tf
import pandas as pd


In [ ]:
cd drive/My\ Drive

/content/drive/My Drive


In [ ]:

knn_from_joblib = joblib.load('models/knn.pkl')  
rf_from_joblib = joblib.load('models/rf.pkl')
network = tf.keras.models.load_model('models/network')

In [ ]:
import tensorflow as tf
import pandas as pd 
test_data = pd.read_csv("test.csv",header=None)
test_img = 'validation'
test_data = test_data.drop([test_data.index[0]])
print(test_data)


                  0           1       2     3                4
1      ISIC_0052060  IP_3579794    male  70.0              NaN
2      ISIC_0052349  IP_7782715    male  40.0  lower extremity
3      ISIC_0058510  IP_7960270  female  55.0            torso
4      ISIC_0073313  IP_6375035  female  50.0            torso
5      ISIC_0073502  IP_0589375  female  45.0  lower extremity
...             ...         ...     ...   ...              ...
10978  ISIC_9992485  IP_4152479    male  40.0            torso
10979  ISIC_9996992  IP_4890115    male  35.0            torso
10980  ISIC_9997917  IP_2852390    male  25.0  upper extremity
10981  ISIC_9998234  IP_8861963    male  65.0  lower extremity
10982  ISIC_9999302  IP_6214039    male  30.0  upper extremity

[10982 rows x 5 columns]


In [ ]:
def columns_to_del(columns,data):
  for column in columns:
    del data[column]
def encode_and_bind(original_dataframe, feature_to_encode):
  dummies = pd.get_dummies(original_dataframe[[feature_to_encode]])
  res = pd.concat([original_dataframe, dummies], axis=1)
  res = res.drop([feature_to_encode], axis=1)
  return(res) 
def preprocess_imageless(dataset):
  res = dataset.copy()
  delete_list = [0, 1] 
  columns_to_del(delete_list,res)
  print(res)
  res = encode_and_bind(res,4)
  print(res.columns)
  res = res[[2, '4_head/neck',3,  '4_torso','4_upper extremity']]  # Removed feature "sepal length"
  return res

res = preprocess_imageless(test_data)
print(res)

            2     3                4
1        male  70.0              NaN
2        male  40.0  lower extremity
3      female  55.0            torso
4      female  50.0            torso
5      female  45.0  lower extremity
...       ...   ...              ...
10978    male  40.0            torso
10979    male  35.0            torso
10980    male  25.0  upper extremity
10981    male  65.0  lower extremity
10982    male  30.0  upper extremity

[10982 rows x 3 columns]
Index([                  2,                   3,       '4_head/neck',
       '4_lower extremity',    '4_oral/genital',     '4_palms/soles',
                 '4_torso', '4_upper extremity'],
      dtype='object')
            2  4_head/neck     3  4_torso  4_upper extremity
1        male            0  70.0        0                  0
2        male            0  40.0        0                  0
3      female            0  55.0        1                  0
4      female            0  50.0        1                  0
5      female

In [ ]:

from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
res[2] = le.fit_transform(res[2])
print(res)

from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
scaler.fit(res)

res = scaler.transform(res)


       2  4_head/neck     3  4_torso  4_upper extremity
1      1            0  70.0        0                  0
2      1            0  40.0        0                  0
3      0            0  55.0        1                  0
4      0            0  50.0        1                  0
5      0            0  45.0        0                  0
...   ..          ...   ...      ...                ...
10978  1            0  40.0        1                  0
10979  1            0  35.0        1                  0
10980  1            0  25.0        0                  1
10981  1            0  65.0        0                  0
10982  1            0  30.0        0                  1

[10982 rows x 5 columns]


In [ ]:
pr_1 = (knn_from_joblib.predict(res))
pr_2 = (rf_from_joblib.predict(res))
pr_3 = (network.predict(res))

In [ ]:
weighted_predictions = list()
pre_len = len(pr_1)
for i in range(0,pre_len):
  p_1 = pr_1[i]
  p_2 = pr_2[i]
  p_3 = pr_3[i]
  weighted_predictions.append( ((p1 * 8 ) + ( p_2 * 7) + (p_3[0] * 6)) / 21 )


In [ ]:
print(test_data)
print((weighted_predictions))
max = weighted_predictions[0]
print(max[0])
for i in weighted_predictions:
  if i[0] > max[0]:
    max = i
print(max[0])    

In [ ]:

dat1 = test_data.copy()
w_p_arr = list()
for i in weighted_predictions:
  w_p_arr.append(i)
w_p_df = pd.DataFrame({'target': w_p_arr})
dat1 = dat1.join(w_p_df)
print(dat1)

                  0           1       2     3                4  target
1      ISIC_0052060  IP_3579794    male  70.0              NaN     0.0
2      ISIC_0052349  IP_7782715    male  40.0  lower extremity     0.0
3      ISIC_0058510  IP_7960270  female  55.0            torso     0.0
4      ISIC_0073313  IP_6375035  female  50.0            torso     0.0
5      ISIC_0073502  IP_0589375  female  45.0  lower extremity     0.0
...             ...         ...     ...   ...              ...     ...
10978  ISIC_9992485  IP_4152479    male  40.0            torso     0.0
10979  ISIC_9996992  IP_4890115    male  35.0            torso     0.0
10980  ISIC_9997917  IP_2852390    male  25.0  upper extremity     0.0
10981  ISIC_9998234  IP_8861963    male  65.0  lower extremity     0.0
10982  ISIC_9999302  IP_6214039    male  30.0  upper extremity     NaN

[10982 rows x 6 columns]


In [ ]:
del dat1[1]
del dat1[2]
del dat1[3]
del dat1[4]
dat1.to_csv("submit.csv")